# Chaveiro 3D Plus (produção, modelos coloridos)

**Para quem é**: makers, designers e pessoas com noções básicas de Python/Jupyter e impressão 3D.

**Pré-requisitos**:
- Python 3.10+
- Jupyter Notebook ou JupyterLab
- Noções básicas de impressão 3D

**Objetivo de produção**:
- Gerar placas/chaveiros 3D com base, contorno e texto em relevo
- Exportar modelos com cores preservadas (GLB/PLY) e 3MF multiobjeto
- Gerar lotes a partir de uma lista ou CSV


## Outline

1. Instalação e imports
2. Configurações e utilitários
3. Geração do modelo 3D
4. Exportação com cores
5. Exemplo mínimo e batch
6. Configuração de produção e validação


In [1]:
# Instalação (execute apenas se necessário)
%pip -q install build123d trimesh numpy networkx lxml
# Opcional para 3MF com cores (depende do sistema)
# %pip -q install py3mf


Note: you may need to restart the kernel to use updated packages.


## Step 1 - Imports e detecção de dependências

Nesta etapa carregamos as bibliotecas e detectamos dependências opcionais.


In [2]:
from __future__ import annotations

import math
import re
import tempfile
import unicodedata
from dataclasses import dataclass
from pathlib import Path

import numpy as np

from build123d import (
    BuildSketch,
    Rectangle,
    Circle,
    Text,
    Align,
    Location,
    Locations,
    extrude,
    fillet,
    offset,
    Mode,
    Compound,
    Color,
    Mesher,
)

try:
    import trimesh
except Exception as e:
    trimesh = None
    print("Aviso: trimesh não disponível -> exportação com cores será limitada.")

try:
    import py3mf  # opcional
except Exception:
    py3mf = None


## Step 2 - Configurações e utilitários

Aqui definimos dimensões, paletas de cor e helpers para nomes de arquivo e cores.


In [3]:
@dataclass
class PlateConfig:
    length: float = 100.0
    width: float = 50.0
    thickness: float = 4.0
    corner_radius: float = 5.0
    hole_radius: float = 2.5
    hole_edge_offset: float = 8.0  # distância da borda esquerda até o centro do furo

@dataclass
class RimConfig:
    width: float = 2.0
    height: float = 1.0

@dataclass
class TextConfig:
    name_size: float = 18.0
    surname_size: float = 10.0
    text_height: float = 2.6
    outline_thickness: float = 0.9
    max_width_margin: float = 6.0
    font: str | None = None  # ex: "Arial" se estiver instalado

COLOR_SCHEMES = {
    "azul_branco": (Color("blue"), Color("white")),
    "vermelho_amarelo": (Color("red"), Color("yellow")),
    "preto_branco": (Color("black"), Color("white")),
    "verde_preto": (Color("green"), Color("black")),
    "branco_preto": (Color("white"), Color("black")),
    "laranja_azul": (Color("orange"), Color("blue")),
}

def slugify(text: str) -> str:
    text = unicodedata.normalize("NFKD", text).encode("ascii", "ignore").decode("ascii")
    text = re.sub(r"[^a-zA-Z0-9]+", "_", text).strip("_")
    return text or "placa"

def color_to_rgba255(color: Color) -> tuple[int, int, int, int]:
    if not isinstance(color, Color):
        color = Color(color)
    tpl = tuple(color)
    if len(tpl) == 3:
        r, g, b = tpl
        a = 1.0
    else:
        r, g, b, a = tpl
    return (int(r * 255), int(g * 255), int(b * 255), int(a * 255))


## Step 3 - Geração do modelo 3D

Funções principais para criar a base, o contorno e o texto em relevo com ajuste automático de tamanho.


In [4]:
def _text_width(text: str, font_size: float, font: str | None) -> float:
    if not text.strip():
        return 0.0
    kwargs = {"font_size": font_size, "align": (Align.CENTER, Align.CENTER)}
    if font:
        kwargs["font"] = font
    with BuildSketch() as sk:
        Text(text, **kwargs)
    return sk.sketch.bounding_box().size.X

def _fit_font_size(text: str, target_width: float, start_size: float, font: str | None, min_size: float = 8.0) -> float:
    size = start_size
    for _ in range(24):
        if _text_width(text, size, font) <= target_width:
            return size
        size *= 0.92
        if size <= min_size:
            return size
    return size

def build_base(plate: PlateConfig, color: Color):
    with BuildSketch() as sk:
        Rectangle(plate.length, plate.width)
        fillet(sk.vertices(), radius=plate.corner_radius)
        hole_x = -plate.length / 2 + plate.hole_edge_offset
        with Locations((hole_x, 0)):
            Circle(radius=plate.hole_radius, mode=Mode.SUBTRACT)
    base = extrude(sk.sketch, amount=plate.thickness)
    base.color = color
    base.label = "Base"
    return base

def build_rim(plate: PlateConfig, rim: RimConfig, color: Color):
    with BuildSketch() as sk:
        Rectangle(plate.length, plate.width)
        fillet(sk.vertices(), radius=plate.corner_radius)
        offset(sk.sketch, amount=-rim.width, mode=Mode.SUBTRACT)
    rim_solid = extrude(sk.sketch, amount=rim.height)
    rim_solid = rim_solid.move(Location((0, 0, plate.thickness)))
    rim_solid.color = color
    rim_solid.label = "Rim"
    return rim_solid

def create_text_parts(
    text: str,
    font_size: float,
    y_pos: float,
    plate: PlateConfig,
    text_cfg: TextConfig,
    color_outline: Color,
    color_face: Color,
):
    if not text or not text.strip():
        return None, None, font_size

    max_width = plate.length - 2 * text_cfg.max_width_margin
    fitted_size = _fit_font_size(text, max_width, font_size, text_cfg.font)

    kwargs = {"font_size": fitted_size, "align": (Align.CENTER, Align.CENTER)}
    if text_cfg.font:
        kwargs["font"] = text_cfg.font

    with BuildSketch() as sk:
        Text(text, **kwargs)

    text_shape = sk.sketch
    input_faces = text_shape.faces() if isinstance(text_shape, Compound) else [text_shape]

    outlines = []
    faces = []
    final_loc = Location((0, y_pos, plate.thickness))

    for face in input_faces:
        try:
            outline_body = extrude(face, amount=text_cfg.text_height)
            inner_body = None

            try:
                inner_sk = offset(face, amount=-abs(text_cfg.outline_thickness))
                inner_faces = inner_sk.faces()
                if inner_faces:
                    inner_body = extrude(inner_faces[0], amount=text_cfg.text_height)
                    outline_body = outline_body - inner_body
            except Exception:
                inner_body = None

            outline_body = outline_body.move(final_loc)
            outline_body.color = color_outline
            outlines.append(outline_body)

            if inner_body:
                inner_body = inner_body.move(final_loc)
                inner_body.color = color_face
                faces.append(inner_body)
        except Exception as e:
            print(f"Aviso: falha ao processar letra '{text}': {e}")

    outline_comp = Compound(children=outlines) if outlines else None
    face_comp = Compound(children=faces) if faces else None

    if outline_comp:
        outline_comp.label = "Text_Outline"
    if face_comp:
        face_comp.label = "Text_Face"

    return outline_comp, face_comp, fitted_size

def generate_nameplate_parts(
    name: str,
    surname: str,
    plate: PlateConfig,
    rim: RimConfig,
    text_cfg: TextConfig,
    color_base: Color,
    color_outline: Color,
):
    parts = {}

    base = build_base(plate, color_base)
    rim_part = build_rim(plate, rim, color_outline)

    parts["base"] = base
    parts["rim"] = rim_part

    if surname and surname.strip():
        name_y = 6.0
        surname_y = -10.0
    else:
        name_y = 0.0
        surname_y = 0.0

    name_outline, name_face, name_size = create_text_parts(
        name, text_cfg.name_size, name_y, plate, text_cfg, color_outline, color_base
    )
    if name_outline:
        parts["name_outline"] = name_outline
    if name_face:
        parts["name_face"] = name_face

    surname_outline, surname_face, surname_size = create_text_parts(
        surname, text_cfg.surname_size, surname_y, plate, text_cfg, color_outline, color_base
    )
    if surname_outline:
        parts["surname_outline"] = surname_outline
    if surname_face:
        parts["surname_face"] = surname_face

    compound = Compound(children=[p for p in parts.values() if p])
    compound.label = "Nameplate"

    meta = {
        "fitted_name_size": name_size,
        "fitted_surname_size": surname_size,
    }

    return parts, compound, meta


## Step 4 - Exportação com cores

A exportação em 3MF multiobjeto continua disponível. Para preservar cores, usamos GLB/PLY e tentamos 3MF com cores quando possível.


In [5]:
def _flatten_shapes(shape):
    shapes = []
    if hasattr(shape, "children") and shape.children:
        for ch in shape.children:
            shapes.extend(_flatten_shapes(ch))
    else:
        shapes.append(shape)
    return shapes

def export_3mf_multiobject(compound, filepath: Path):
    exporter = Mesher()
    for solid in _flatten_shapes(compound):
        exporter.add_shape(solid)
    exporter.write(str(filepath))


def export_colored_scene(parts: dict, color_map: dict[str, Color], out_path: Path, file_type: str):
    if trimesh is None:
        return False
    try:
        import networkx as _nx  # required by trimesh 3MF export
    except Exception:
        return False
    try:
        import lxml  # required by trimesh 3MF export
    except Exception:
        return False

        raise RuntimeError("trimesh não está instalado. Instale para exportar cores.")

    scene = trimesh.Scene()

    with tempfile.TemporaryDirectory() as td:
        td = Path(td)
        for name, shape in parts.items():
            stl_path = td / f"{name}.stl"
            mesher = Mesher()
            mesher.add_shape(shape)
            mesher.write(str(stl_path))

            mesh = trimesh.load_mesh(stl_path, process=False)
            if isinstance(mesh, trimesh.Scene):
                mesh = trimesh.util.concatenate(mesh.dump())

            rgba = np.array(color_to_rgba255(color_map[name]), dtype=np.uint8)
            mesh.visual.vertex_colors = np.tile(rgba, (len(mesh.vertices), 1))
            scene.add_geometry(mesh, geom_name=name)

    scene.export(str(out_path), file_type=file_type)


def export_3mf_colored_best_effort(parts: dict, color_map: dict[str, Color], out_path: Path) -> bool:
    if trimesh is None:
        return False
    try:
        import networkx as _nx  # required by trimesh 3MF export
    except Exception:
        return False
    try:
        import lxml  # required by trimesh 3MF export
    except Exception:
        return False

        return False
    try:
        export_colored_scene(parts, color_map, out_path, file_type="3mf")
        return True
    except Exception as e:
        print(f"Aviso: 3MF colorido falhou ({e}). Usando 3MF multiobjeto sem cor.")
        return False


## Step 5 - Exemplo mínimo e batch


In [6]:
plate_cfg = PlateConfig()
rim_cfg = RimConfig()
text_cfg = TextConfig()

color_base, color_outline = COLOR_SCHEMES["azul_branco"]

parts, compound, meta = generate_nameplate_parts(
    name="ARTHUR",
    surname="PEREIRA",
    plate=plate_cfg,
    rim=rim_cfg,
    text_cfg=text_cfg,
    color_base=color_base,
    color_outline=color_outline,
)

out_dir = Path("output/nameplates")
out_dir.mkdir(parents=True, exist_ok=True)

base_filename = slugify("Andre Pereira")

# 1) 3MF multiobjeto (compatível com slicers)
export_3mf_multiobject(compound, out_dir / f"{base_filename}.3mf")

# 2) GLB com cores preservadas (visualização)
export_colored_scene(parts, {k: (color_outline if "outline" in k or k == "rim" else color_base) for k in parts}, out_dir / f"{base_filename}.glb", file_type="glb")

# 3) Tenta 3MF colorido (se suportado)
export_3mf_colored_best_effort(parts, {k: (color_outline if "outline" in k or k == "rim" else color_base) for k in parts}, out_dir / f"{base_filename}_color.3mf")

print("Arquivos gerados em:", out_dir)


Arquivos gerados em: output/nameplates


## Step 6 - Lote (lista ou CSV)

Abaixo, um exemplo de geração em lote a partir de uma lista. Você pode trocar por CSV facilmente.


In [ ]:
plates = [
    {"name": "Ana", "surname": "Silva"},
    {"name": "Carlos", "surname": "Lima"},
    {"name": "João", "surname": ""},
]

color_base, color_outline = COLOR_SCHEMES["branco_preto"]

for p in plates:
    name = p["name"]
    surname = p.get("surname", "")

    parts, compound, meta = generate_nameplate_parts(
        name=name,
        surname=surname,
        plate=plate_cfg,
        rim=rim_cfg,
        text_cfg=text_cfg,
        color_base=color_base,
        color_outline=color_outline,
    )

    base_filename = slugify(f"{name}_{surname}")

    export_3mf_multiobject(compound, out_dir / f"{base_filename}.3mf")
    export_colored_scene(parts, {k: (color_outline if "outline" in k or k == "rim" else color_base) for k in parts}, out_dir / f"{base_filename}.glb", file_type="glb")

print("Lote concluído.")


## Configuração de produção e validação

Use os parâmetros abaixo para garantir legibilidade na impressão. Esta seção implementa o aumento de espessura do texto sugerido anteriormente.


In [ ]:
# Configuração de produção (texto mais espesso)
prod_plate = PlateConfig(length=100.0, width=50.0, thickness=4.0)
prod_rim = RimConfig(width=2.0, height=1.0)
prod_text = TextConfig(
    text_height=2.8,          # mais alto para melhor leitura
    outline_thickness=1.0,    # contorno mais espesso
    name_size=18.0,
    surname_size=10.0,
)

parts, compound, meta = generate_nameplate_parts(
    name="Produção",
    surname="Teste",
    plate=prod_plate,
    rim=prod_rim,
    text_cfg=prod_text,
    color_base=Color("white"),
    color_outline=Color("black"),
)

export_3mf_multiobject(compound, out_dir / "producao_teste.3mf")
export_colored_scene(parts, {k: (Color("black") if "outline" in k or k == "rim" else Color("white")) for k in parts}, out_dir / "producao_teste.glb", file_type="glb")

print("Validação rápida concluída.")


## Pitfalls e extensões

- 3MF colorido depende do suporte do slicer. Use GLB/PLY para visualizar cores com fidelidade.
- Fontes variam por sistema. Se a fonte não existir, o build123d pode usar um fallback.
- Textos muito longos podem ficar pequenos; ajuste `max_width_margin` e `name_size`.

Extensão sugerida: adicionar leitura de CSV (colunas `name` e `surname`) e um modo de pré-visualização com `ocp_vscode`.
